# Part I. Preparing a Docker Image

Before diving into the nitty-gritty of Sagemaker training and deploy, it is crutial to make sure the training and deploy "container" is set up. This container will provide the most up-to-date version of GluonCV, MXNet and other essential programming environments, which enable us to achieve state-of-the-art(SOTA) model training and deployment.
Let's take a look of the process of setting up a container.

### 1. Building a Docker Image

If this is the first time of using SageMaker training and deployment, you will need to prepare a Docker image by running the following commands:

In [1]:
import os
os.chdir("./container/") ## Change the working directory to `container`
os.getcwd()

!bash build_and_push.sh my-repo image_classification


An error occurred (AccessDeniedException) when calling the SetRepositoryPolicy operation: User: arn:aws:sts::058295922468:assumed-role/AmazonSageMaker-ExecutionRole-20200409T121891/SageMaker is not authorized to perform: ecr:SetRepositoryPolicy on resource: arn:aws:ecr:us-east-1:058295922468:repository/my-repo
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  56.32kB
Step 1/11 : FROM ubuntu:18.04
 ---> c3c304cb4f22
Step 2/11 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> 579719c7bceb
Step 3/11 : ARG APP=image_classification
 ---> Using cache
 ---> 09f8b0cca80c
Step 4/11 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget        

### 2. Granting the ECR Repo Access

Since Amazon ECR repository policies are a subset of IAM policies that are scoped for, and specifically used for, controlling access to individual Amazon ECR repositories. IAM policies are generally used to apply permissions for the entire Amazon ECR service but can also be used to control access to specific resources as well. Amazon ECR requires that users have allow permissions to the ecr:GetAuthorizationToken API through an IAM policy before they can authenticate to a registry and push or pull any images from any Amazon ECR repository. More details: https://docs.aws.amazon.com/AmazonECR/latest/userguide/repository-policies.html

We can go to the [ECR Repo](https://console.aws.amazon.com/ecr/repositories/my_repo/permissions?region=us-east-1) to grant permission to access the repo with the permission like the following.

```{`json}
{
  "Version": "2008-10-17",
  "Statement": [
    {
      "Sid": "All-Allow",
      "Effect": "Allow",
      "Principal": {
        "AWS": [
          "arn:aws:iam::058295922468:user/rlhu",
          "arn:aws:sts::058295922468:assumed-role/AmazonSageMaker-ExecutionRole-20200409T121891/SageMaker"
        ]
      },
      "Action": "*"
    }
  ]
}
```

### 3. Pushing the Docker image

Now with the access permission to the new ECR repo, let's push Docker image by calling the `push.sh` script. 

In [3]:
!bash push.sh my-repo image_classification

The push refers to repository [058295922468.dkr.ecr.us-east-1.amazonaws.com/my-repo]

2baed6f7: Preparing 
56b4faa1: Preparing 
fae1e837: Preparing 
af84388f: Preparing 
7458d04b: Preparing 
37a24627: Preparing 
ef4a95c3: Preparing 
56b4faa1: Pushed   540.4MB/536MBMBPushing  284.9MB/536MBPushing  541.9MBlatest: digest: sha256:2d13037f5d7a912f2012a444d9d2ad54e4e05862903d6768d677fb3f919c1d1a size: 1992


# Part 2: Training, Infernce and Deployment


## 1. Training

Once we have the container packaged, you can use it to train and serve models. Let's do that with the algorithm we made above.

### Setting up the Environment

Here we specify a bucket to use and the role that will be used for working with Amazon SageMaker.

In [4]:
os.chdir("../") ## Change the working directory back to main

from sagemaker import get_execution_role
role = get_execution_role()

### Creating the Session

The session remembers our connection parameters to Amazon SageMaker. We'll use it to perform all of our SageMaker operations.

In [5]:
import sagemaker as sage

sess = sage.Session()

### Defining the account, region and ECR address


In [6]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
ecr_name = "my-repo"
ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, ecr_name)

### Uploading Training Data

We can upload the training data to the corresponding S3 bucket: https://s3.console.aws.amazon.com/s3/buckets/sagemaker-us-east-1-058295922468/sagemaker-deploy-gluoncv/data/?region=us-east-1

In [10]:
s3_bucket = "sagemaker-deploy-gluoncv"
# model_path = "s3://{}/{}/model".format(sess.default_bucket(), s3_bucket)
# os.path.join(model_path, "model.tar.gz")
# model_prefix = s3_bucket + "/model"
train_data_local = "./data/minc-2500/train"
train_data_dir_prefix = s3_bucket + "/data/train"


# model_local_path = "model_output"
train_data_upload = sess.upload_data(path=train_data_local, 
#                                 bucket=s3_bucket, 
                                key_prefix=train_data_dir_prefix)
print("Train input uploaded to " + train_data_upload)

Train input uploaded to s3://sagemaker-us-east-1-058295922468/sagemaker-deploy-gluoncv/data/train


### Creating an Estimator

In order to use Amazon SageMaker to fit our algorithm, we'll create an `Estimator` that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

* The __role__ is defined as above.
* The __session__ is the SageMaker session object that we defined above.
* The __image name__ is the name of ECR image we created above.
* The __training instance type__ which is the type of machine to use for training.
* The __training instance count__ which is the number of machines to use for training.
* The __output path__ determines where the model artifact will be written.

In [8]:
from sagemaker.estimator import Estimator

train_dir = "data/minc-2500/train"
hyperparameters = {'epochs': 1, 
                   'model_name': 'resnet18_v1b'}
instance_type = 'ml.c4.2xlarge'  # 'ml.p2.xlarge'
s3_path = "s3://{}/{}/model".format(sess.default_bucket(), s3_bucket)
model_path = os.path.join(s3_path, "model.tar.gz")
print(model_path)

s3://sagemaker-us-east-1-058295922468/sagemaker-deploy-gluoncv/model/model.tar.gz


### Fitting the Estimator

Then we call the `fit()` function on the estimator to train against the data that we uploaded above.

In [ ]:
classifier = Estimator(role=role, 
                       sagemaker_session=sess,
                       image_name=ecr_image, 
                       train_instance_count=1,
                       train_instance_type=instance_type,
                       hyperparameters=hyperparameters,
#                        checkpoint_local_path="model_output/", 
                       output_path=s3_path
                       )
# train_data_upload = model_upload
classifier.fit(train_data_upload)

2020-05-19 19:26:52 Starting - Starting the training job...
2020-05-19 19:26:55 Starting - Launching requested ML instances....

## 2.Batch Inferencing

After our model has been trained, we simply use a demo image for testing our model. We first upload this image the S3 bucket and we can test the model after deplyment.

In [ ]:
demo_dir = "data/demo"
test_image = "cat1.jpg"
sample_inference_input_prefix = s3_bucket + "/data/test"

demo_input = sess.upload_data(os.path.join(demo_dir, test_image), 
                                   key_prefix=sample_inference_input_prefix) 
print("Demo input uploaded to " + demo_input)

##  3. Deploying the Model

Deploying the model to Amazon SageMaker hosting just requires a `deploy` call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

Note that deployment takes a little bit longer than all the previous steps.

In [ ]:
# from sagemaker.predictor import csv_serializer

model = classifier.create_model()
predictor = classifier.deploy(1, 'ml.m4.xlarge')

### Choose some data and use it for a prediction

In order to do some predictions, we'll use a demo jpeg image to test the model.

In [ ]:
with open(os.path.join(demo_dir, test_image), 'rb') as f:
    x = f.read()
    print(predictor.predict(x, initial_args={'ContentType':'image/jpeg'}).decode('utf-8'))

### Cleanup Endpoint

When you're done with the endpoint, you'll want to clean it up.

In [ ]:
sess.delete_endpoint(predictor.endpoint)